In [1]:
from pyvis.network import Network
import networkx as nx
import pandas as pd
import matplotlib.pyplot as plt
import pickle
import bs4
from os import listdir
from os.path import isfile, join

%matplotlib inline

In [2]:
docs = ["Issue and Listing of Non-Convertible Redeemable Preference Shares", "Investment Advisers", "Depositories and Participants", "Mutual Funds", "Employees Service", "Substantial Acquisition of Shares and Takeovers", "Appointment of Administrator and Procedure for Refunding to the Investors", "Prohibition of Fraudulent and Unfair Trade Practices relating to Securities Market", "Know Your Client", "Prohibition of Insider Trading", "Merchant Bankers", "Issue and Listing of Securitised Debt Instruments and Security Receipts", "Delisting of Equity Shares", "Issue of Capital and Disclosure Requirements", "Foreign Venture Capital Investor", "Procedure for Board Meetings", "Custodian", "Ombudsman", "Investor Protection and Education Fund", "Foreign Portfolio Investors", "Issue of Sweat Equity", "Collective Investment Scheme", "Portfolio Managers", "Research Analysts", "Procedure for Search and Seizure", "Issue Of Capital And Disclosure Requirements", "Share Based Employee Benefits", "Debenture Trustees", "Alternative Investment Funds", "Stock Exchanges and Clearing Corporations", "Self Regulatory Organisations", "Settlement Proceedings", "Issue and Listing of Municipal Debt Securities", "Buy-back of Securities", "Issue and Listing of Debt Securities", "Infrastructure Investment Trusts", "Stock Brokers", "Listing Obligations and Disclosure Requirements", "Registrars to an Issue and Share Transfer Agents", "Real Estate Investment Trusts", "Intermediaries", "Certification of Associated Persons in the Securities Markets", "Credit Rating Agencies", "Regulatory Fee on Stock Exchanges", "Underwriters", "Buy Back Of Securities", "Bankers to an Issue", "Central Database of Market Participants"]
with open('cleanedregulations48.pkl','rb') as f:
    finalclean48 = pickle.load(f)  

#### Extracting cross-references

In [3]:
def regids(k):
    regind = []
    for i in k:
        p = i.split()[0]
        if('.' in p):
            if(len(p) < 4):
                l = p.replace('.','')
                regind.append([l,'1'])
            else:
                r = regind[-1]
                r1 = int(r[1]) + 1
                regind.append([l,str(r1)])
        else:
            t=0
            while(t<len(p)):
                if(p[t] == '('):
                    start = t
                if(p[t] == ')'):
                    break
                t+=1
            start = start+1
            sind = ''
            while(start<t):
                sind = sind + p[start]
                start+=1
        
            regind.append([l,sind])
    return regind

In [4]:
def crossref(k, regind, reggraphdata):
    total=0
    for i in k:
        totalreg = []
        refregs = [] # [subreg id, reg id]
        if('sub-regulation (' in i):
            p = i.split()
            t=0
            start = []
            end = []
            totalreg = []
            while(t<len(p)):
                if(p[t]=='sub-regulation'):
                    if(len(start) != len(end)):
                        end.append('nil')
                    start.append(t)
                if(p[t]=='regulation'):
                    if(len(start) == len(end)+1):
                        end.append(t)
                    else:
                        if(t < len(p)-1):
                            cleanreg = p[t+1]
                            if('.' in p[t+1]):
                                cleanreg = p[t+1].replace('.','')
                            if(',' in p[t+1]):
                                cleanreg = p[t+1].replace(',','')
                            if(cleanreg not in totalreg):
                                totalreg.append(cleanreg)
                if(len(start) !=0):
                    if(t < len(p)-5 and t!=len(p)-1):
                        if(t == start[-1]+4):
                            if(len(start) != len(end)):
                                end.append('nil')
                    else:
                        if(t == len(p)-1):
                            if(len(start) != len(end)):
                                end.append('nil')
                t+=1
            '''
            print(start)
            print(end)
            print(len(p))
            print(i)
            print('-----------------')
            '''
            
            itert=0
            while(itert<len(start) and start[itert]<len(p)-1):
                m = p[start[itert]+1]
            
                t=0
                while(t<len(m)):
                    if(m[t] == '('):
                        mstart = t
                    if(m[t] == ')'):
                        break
                    t+=1
                mstart = mstart+1
                sind = ''
                while(mstart<t):
                    sind = sind + m[mstart]
                    mstart+=1
                
                if(end[itert] != 'nil'):
                    if(end[itert] < len(p)-1):
                        cleanreg = p[end[itert]+1]
                        if('.' in p[end[itert]+1]):
                            cleanreg = p[end[itert]+1].replace('.','')
                        if(',' in p[end[itert]+1]):
                            cleanreg = p[end[itert]+1].replace(',','')
                        if([sind,cleanreg] not in refregs):
                            refregs.append([cleanreg,sind])
                            if(cleanreg in totalreg):
                                totalreg.remove(cleanreg)
                    else:
                        start.pop(itert)
                else:
                    r = regind[total]
                    if([r[0],sind] not in refregs):
                        refregs.append([r[0],sind])
                itert+=1
                
            for j in totalreg:
                riter=0
                for r in regind:
                    if(j in r[0]):
                        if([k[riter],i, '0'] not in reggraphdata):
                            reggraphdata.append([k[riter],i,'0'])
                    riter = riter+1
            
            for refdata in refregs:
                if(refdata in regind):
                    sourceind = regind.index(refdata)
                    if([k[sourceind],i, '0'] not in reggraphdata):
                        reggraphdata.append([k[sourceind],i, '0'])
        total = total+1
    return reggraphdata

In [10]:
reggraphdata = []
t=0
for k in finalclean48:
    regind = regids(k)
    reggraphdata = crossref(k, regind, reggraphdata)
    t=t+1

#### Chapter wise

In [5]:
with open('docs_chaps.pkl','rb') as f:
    docchaps = pickle.load(f)

In [6]:
def chapterconnect(k, d, reggraphdata):
    
    indlist = []
    t=1
    p=t
    while(t<len(d)):
        if(d[t] != d[p]):
            p=t
            indlist.append(t-1)
        if(t==len(d)-1):
            indlist.append(t)
        t+=1
    
    t=0
    for i in indlist:
        while(t<i):
            if([k[t],k[t+1],'0'] not in reggraphdata and [k[t],k[t+1],'1'] not in reggraphdata):
                reggraphdata.append([k[t],k[t+1],'1'])
            t+=1
    
    return reggraphdata

In [11]:
citer=0
for k in finalclean48:
    d = docchaps[citer]
    reggraphdata = chapterconnect(k, d, reggraphdata)
    citer = citer+1

#### Common topic words

In [7]:
with open('reg_topics.pkl','rb') as f:
    regtopics = pickle.load(f)

In [8]:
regulations = []
for i in finalclean48:
    regulations.extend(i)

In [9]:
def commontopics(regtopics,regulations,reggraphdata):
    t=0
    while(t<len(regtopics)):
        p=t+1
        while(p<len(regtopics)):
            for i in regtopics[t]:
                if(i in regtopics[p]):
                    if([regulations[t],regulations[p],'0'] not in reggraphdata and [regulations[t],regulations[p],'1'] not in reggraphdata and [regulations[t],regulations[p],'2'] not in reggraphdata):
                        reggraphdata.append([regulations[t],regulations[p],'2'])
                        break
            p+=1
        t+=1
    return reggraphdata

In [12]:
reggraphdata = commontopics(regtopics,regulations,reggraphdata)

KeyboardInterrupt: 

In [ ]:
import pickle
with open('reggraphdata.pkl', 'wb') as f:
    pickle.dump(reggraphdata, f)

#### Knowledge Graph Construction

#### Layer 1

In [13]:
def layer1(df,h):
    sources = df['source']
    targets = df['target']
    labels = df['label']
    q = 'Layer 1 - Regulatory Documents'
    edge_data = zip(sources, targets, labels)
    
    #nt = Network(height="750px", width="100%", heading=q)
    nt = Network(height="100%", width="100%",heading=q)
    nt.hrepulsion()
    
    t=0
    for e in edge_data:
        src = e[0]
        dst = e[1]
        lbl = e[2]
        
        if(h[t] == 'amendment'):
            nt.add_node(src, src, title=src, color = "blue", size=100, url='./Regdoc_graphs/'+src+'.html')
            nt.add_node(dst, label='a', title='Amendment Timeline of '+src, color = "red",size=50, url='./amendment_timeline/'+src+'.html')
            nt.add_edge(src, dst, value=50, title=lbl, color = "#000000", length=700)
        else:
            nt.add_node(src, src, title=src, color = "blue", size=100,url='./Regdoc_graphs/'+src+'.html')
            nt.add_node(dst, dst, title=dst, color = "blue", size=100,url='./Regdoc_graphs/'+dst+'.html')
            nt.add_edge(src, dst, value=50, title=lbl, color = "#000000", length=700)
        t=t+1
    
    nt.show_buttons()  
    nt.show('layer1.html')
    
    with open("layer1.html") as inf:
        txt = inf.read()
        soup = bs4.BeautifulSoup(txt)  
    c = soup.find_all('script',type="text/javascript")
    tag = c[-1]

    t=0
    start = 0
    tindex = []
    tokens1 = tag.contents[0].splitlines(True)

    t=0
    appendind = []
    for i in tokens1:
        if('return' in i):
            appendind.append(t)
        t=t+1
    
    with open("/home/deepti-saravanan/Desktop/laws/interactive.js") as f:
        jstext = f.read()
    
    p = []
    t=0
    while(t<appendind[-1]):
        p.append(tokens1[t])
        t=t+1
    p.append(jstext)
    t = appendind[-1]
    while(t<len(tokens1)):
        p.append(tokens1[t])
        t=t+1
    
    s = ''
    for i in p:
        if(i == '\n'):
            s = s + ' ' + '\n'
        else:
            s = s + ' ' + i
        
    tag.string = s

    soup.find_all('script',type="text/javascript")[-1] = tag
    with open("layer1.html", "w") as outf:
        outf.write(str(soup))
        
    with open("layer1.html") as inf:
        txt = inf.read()
        soup = bs4.BeautifulSoup(txt)  
    c = soup.find_all('script',type="text/javascript")
    tag = c[-1]
    
    t=0
    start = 0
    tindex = []
    tokens1 = tag.contents[0].splitlines(True)
    
    t0=0
    while(t0<len(tokens1)):
        if('"edges": {' in tokens1[t0]):
            break
        t0+=1
        
    with open("/home/deepti-saravanan/Desktop/laws/nodefont.js") as f:
        jstext = f.read()
    
    p = []
    t=0
    while(t<t0):
        p.append(tokens1[t])
        t=t+1
    p.append(jstext)
    t = t0
    while(t<len(tokens1)):
        p.append(tokens1[t])
        t=t+1
    
    s = ''
    for i in p:
        if(i == '\n'):
            s = s + ' ' + '\n'
        else:
            s = s + ' ' + i
        
    tag.string = s
    
    soup.find_all('script',type="text/javascript")[-1] = tag
    with open("layer1.html", "w") as outf:
        outf.write(str(soup))

In [189]:
layerinfo = []
h=[]
t=0
for i in docs:
    if(t<len(docs)-1):
        layerinfo.append([i,docs[t+1],'Click on the nodes for detailed analysis of regulations'])
        h.append('regulations')
    layerinfo.append([i,'a'+str(t),'Detailed biography of regulations'])
    h.append('amendment')
    t=t+1
layerinfo.append([docs[-1],'all','Click on the nodes for detailed analysis of regulations'])
h.append('regulations')
df = pd.DataFrame(layerinfo, columns=['source','target','label'])
layer1(df,h)

#### Layer 2 (Amendments)

#### Concept Papers

In [14]:
def conceptpaper(docs):
    
    conreg_dict = {}
    for i in docs:
        conreg_dict[i.lower()] = i.lower()
    conreg_dict['aif'] = 'alternative investment funds'
    conreg_dict['aifs'] = 'alternative investment funds'
    conreg_dict['secc'] = 'stock exchanges and clearing corporations'
    conreg_dict['reit'] = 'real estate investment trusts'
    conreg_dict['reits'] = 'real estate investment trusts'
    conreg_dict['invit'] = 'infrastructure investment trusts'
    conreg_dict['invits'] = 'infrastructure investment trusts'
    conreg_dict['pms'] = 'portfolio managers'
    conreg_dict['icdr'] = 'issue of capital and disclosure requirements'
    conreg_dict['fvci'] = 'foreign venture capital investor'
    conreg_dict['fvcis'] = 'foreign venture capital investor'
    conreg_dict['fpi'] = 'foreign portfolio investors'
    conreg_dict['sast'] = 'substantial acquisition of shares and takeovers'
    conreg_dict['ilds'] = 'issue and listing of debt securities'
    conreg_dict['takeover regulations'] = 'substantial acquisition of shares and takeovers'
    conreg_dict['br'] = 'bankers to an issue'
    
    regs = list(conreg_dict.keys())
    onlyfiles = [f for f in listdir('/home/deepti-saravanan/Documents/Deepti/') if isfile(join('/home/deepti-saravanan/Documents/Deepti/',f))]
    
    concept_dir = '/home/deepti-saravanan/Documents/Deepti/'
    regwords = ['regulation', 'Regulation', 'Regulations', 'regulations']
    stopwords = ['background','objective','1.','1.0','/','^','scenario']
    symbols = ['(',')',',','.',]
    flag=0
    final_content = []
    for filename in onlyfiles:
        reg_doc = []
        year = []
        with open(concept_dir+filename, encoding='utf-8', errors='ignore') as f:
            content = f.readlines()
            content = [x.strip() for x in content] 
        for line in content:
            for j in regwords:
                if(j in line):
                    for symb in symbols:
                        line = line.replace(symb,'')
                    for regdocname in regs:
                        if(regdocname in line.lower()):
                            if(conreg_dict[regdocname] not in reg_doc):
                                reg_doc.append(conreg_dict[regdocname])
                                flag=1
                                year.append('2021')
                                for sline in line.split():
                                    if(sline.isdigit() == True and len(sline) == 4):
                                        year[-1] = sline
                            else:
                                ind = reg_doc.index(conreg_dict[regdocname])
                                y = year[ind]
                                if(y == '2021'):
                                    for sline in line.split():
                                        if(sline.isdigit() == True and len(sline) == 4):
                                            year[ind] = sline    
                                        
        if(flag==1):
            flag=0
            c = content[0:3]
            s = ''
            for i in c:
                stopflag=0
                for j in stopwords:
                    if(j in i.lower()):
                        stopflag=1
                        break
                if(stopflag==0):
                    if(len(i.split())<20):
                        s = s + ' ' + i
            kiter=0
            for docnameiter in reg_doc:
                if(year[kiter] == '2021'):
                    for symb in symbols:
                        d = s.replace(symb,'')
                    for sline in d.split():
                        if(sline.isdigit() == True and len(sline) == 4):
                            year[kiter] = sline
                        
                final_content.append([docnameiter,year[kiter],filename,s])
                kiter = kiter+1
            s = ''
    return final_content

In [178]:
final_content = conceptpaper(docs)
with open('conceptpaper_graph.pkl', 'wb') as f:
    pickle.dump(final_content, f)

In [15]:
with open('conceptpaper_graph.pkl', 'rb') as f:
    final_content = pickle.load(f)

#### Linking Amendments Timeline and Concept Papers

In [18]:
def layer20(amendfile):
    
    with open('./amendment/'+amendfile,'rb') as f:
        a0 = pickle.load(f)
        
    adocs = []
    for i in a0:
        if(i[0] not in adocs):
            adocs.append(i[0])
        if(i[1] not in adocs):
            adocs.append(i[1])
            
    cplabels = []
    for i in cp:
        if(i[0] == amendfile.replace('.pkl','').lower()):
            for j in adocs:
                if(i[1] in j):
                    a0.append([j,i[2]])
                    cplabels.append(i[3])
                    break
    a = []
    t=0
    for i in a0:
        if('.txt' in i[1]):
            i.extend(['1',cplabels[t]])
            t=t+1
            a.append(i)
        else:
            i.extend(['0','amendment'])
            a.append(i)
    df = pd.DataFrame(a, columns = ['source','target','h','label'])

    sources = df['source']
    targets = df['target']
    labels = df['label']
    h = df['h']
    q = 'Layer 2 - Amendment Timeline of ' + amendfile.replace('.pkl','')
    edge_data = zip(sources, targets, labels,h)
    
    #nt = Network(height="750px", width="100%", heading=q)
    nt = Network(height="100%", width="100%",heading=q)
    nt.hrepulsion()
    
    for e in edge_data:
        src = e[0]
        dst = e[1]
        lbl = e[2]
        h = e[3]
        
        if(h == '0'):
            nt.add_node(src, src, title=src, color = "blue")
            nt.add_node(dst, dst, title=dst, color = "blue")
            nt.add_edge(src, dst, title=lbl, color = "#000000")
        else:
            nt.add_node(src, src, title=src, color = "blue")
            nt.add_node(dst, dst, title=dst, color = "red",url='file:///home/deepti-saravanan/Documents/reports_for_public_comments/'+dst.replace('.txt','')+'.pdf')
            nt.add_edge(src, dst, title=lbl, color = "#000000")
        t=t+1
    
    nt.show_buttons()
    name = amendfile.replace('.pkl','')
    nt.show('./amendment_timeline/'+name+'.html')
    
    with open('./amendment_timeline/'+name+'.html') as inf:
        txt = inf.read()
        soup = bs4.BeautifulSoup(txt)  
    c = soup.find_all('script',type="text/javascript")
    tag = c[-1]

    t=0
    start = 0
    tindex = []
    tokens1 = tag.contents[0].splitlines(True)

    t=0
    appendind = []
    for i in tokens1:
        if('return' in i):
            appendind.append(t)
        t=t+1
    
    with open("interactive.js") as f:
        jstext = f.read()
    
    p = []
    t=0
    while(t<appendind[-1]):
        p.append(tokens1[t])
        t=t+1
    p.append(jstext)
    t = appendind[-1]
    while(t<len(tokens1)):
        p.append(tokens1[t])
        t=t+1
    
    s = ''
    for i in p:
        if(i == '\n'):
            s = s + ' ' + '\n'
        else:
            s = s + ' ' + i
        
    tag.string = s

    soup.find_all('script',type="text/javascript")[-1] = tag
    with open('./amendment_timeline/'+name+'.html', "w") as outf:
        outf.write(str(soup))
        
    with open('./amendment_timeline/'+name+'.html') as inf:
        txt = inf.read()
        soup = bs4.BeautifulSoup(txt)  
    c = soup.find_all('script',type="text/javascript")
    tag = c[-1]
    
    t=0
    start = 0
    tindex = []
    tokens1 = tag.contents[0].splitlines(True)
    
    t0=0
    while(t0<len(tokens1)):
        if('"edges": {' in tokens1[t0]):
            break
        t0+=1
        
    with open("nodefont.js") as f:
        jstext = f.read()
    
    p = []
    t=0
    while(t<t0):
        p.append(tokens1[t])
        t=t+1
    p.append(jstext)
    t = t0
    while(t<len(tokens1)):
        p.append(tokens1[t])
        t=t+1
    
    s = ''
    for i in p:
        if(i == '\n'):
            s = s + ' ' + '\n'
        else:
            s = s + ' ' + i
        
    tag.string = s
    
    soup.find_all('script',type="text/javascript")[-1] = tag
    with open('./amendment_timeline/'+name+'.html', "w") as outf:
        outf.write(str(soup))

In [19]:
amendfiles = [f for f in listdir('./amendment/') if isfile(join('./amendment/',f))]
with open('conceptpaper_graph.pkl','rb') as f:
    cp = pickle.load(f)
for amendfile in amendfiles:
    layer20(amendfile)

In [41]:
idoc

'Prohibition of Insider Trading'

#### Layer 2 (Reg Docs)

In [53]:
def layer21(df,idoc, regind, igl, lcl, t, cur_regs):
    docnumber = t
    sources = df['source']
    targets = df['target']
    labels = df['label']
    q = 'Layer 2 - ' + idoc
    edge_data = zip(sources, targets, labels)
    
    #nt = Network(height="750px", width="100%", heading=q)
    nt = Network(height="100%", width="100%",heading=q)
    nt.hrepulsion()
    
    lbl = ['Cross-reference', 'Chapterwise', 'Shared topics']
    t=0
    igcount=0
    lccount=0
    for e in edge_data:
        src = e[0]
        dst = e[1]
        h = e[2]
        if(h == '0'):
            sid = cur_regs.index(src)
            did = cur_regs.index(dst)
            nt.add_node(src, label=str(docnumber)+'_'+str(sid), title=src, color = "blue", url='./Reg_graphs/'+str(docnumber)+'_'+str(sid)+'.html')
            nt.add_node(dst, label=str(docnumber)+'_'+str(did), title=dst, color = "blue", url='./Reg_graphs/'+str(docnumber)+'_'+str(did)+'.html')
            nt.add_edge(src, dst, title=lbl[0], value=8, color = "#000000", length=400)
        elif(h == '1'):
            sid = cur_regs.index(src)
            did = cur_regs.index(dst)
            nt.add_node(src, label=str(docnumber)+'_'+str(sid), title=src, color = "blue", url='./Reg_graphs/'+str(docnumber)+'_'+str(sid)+'.html')
            nt.add_node(dst, label=str(docnumber)+'_'+str(did), title=dst, color = "blue", url='./Reg_graphs/'+str(docnumber)+'_'+str(did)+'.html')
            nt.add_edge(src, dst, title=lbl[1], value=7, color = "red", length=400)
        elif(h == '2'):
            sid = cur_regs.index(src)
            did = cur_regs.index(dst)
            nt.add_node(src, label=str(docnumber)+'_'+str(sid), title=src, color = "blue", url='./Reg_graphs/'+str(docnumber)+'_'+str(sid)+'.html')
            nt.add_node(dst, label=str(docnumber)+'_'+str(did), title=dst, color = "blue", url='./Reg_graphs/'+str(docnumber)+'_'+str(did)+'.html')
            nt.add_edge(src, dst, title=lbl[2], color = "green", length=400)
        elif(h == '3'):
            sid = cur_regs.index(src)
            nt.add_node(src, label=str(docnumber)+'_'+str(sid), title=src, color = "blue", url='./Reg_graphs/'+str(docnumber)+'_'+str(sid)+'.html')
            nt.add_node(dst, label='informal_guidance', title=dst, color = "red", url='file:///home/deepti-saravanan/Desktop/knowledge_graph_SEBI/informal_guidance/'+dst.replace('.docx','')+'.pdf')
            nt.add_edge(src, dst, title=igl[igcount], color = "#000000", length=400)
            igcount = igcount+1
        else:
            sid = cur_regs.index(src)
            nt.add_node(src, label=str(docnumber)+'_'+str(sid), title=src, color = "blue", url='./Reg_graphs/'+str(docnumber)+'_'+str(sid)+'.html')
            nt.add_node(dst, label='legal_case', title=dst, color = "black", url='file:///home/deepti-saravanan/Desktop/knowledge_graph_SEBI/case_pdf/'+dst+'.pdf')
            nt.add_edge(src, dst, title=lcl[lccount], color = "#000000", length=400)
            lccount = lccount+1
        t=t+1
    
    nt.show_buttons()  
    nt.write_html('./Regdoc_graphs/'+idoc+'.html')
    
    with open('./Regdoc_graphs/'+idoc+'.html') as inf:
        txt = inf.read()
        soup = bs4.BeautifulSoup(txt)  
    c = soup.find_all('script',type="text/javascript")
    tag = c[-1]

    t=0
    start = 0
    tindex = []
    tokens1 = tag.contents[0].splitlines(True)

    t=0
    appendind = []
    for i in tokens1:
        if('return' in i):
            appendind.append(t)
        t=t+1
    
    with open("interactive.js") as f:
        jstext = f.read()
    
    p = []
    t=0
    while(t<appendind[-1]):
        p.append(tokens1[t])
        t=t+1
    p.append(jstext)
    t = appendind[-1]
    while(t<len(tokens1)):
        p.append(tokens1[t])
        t=t+1
    
    s = ''
    for i in p:
        if(i == '\n'):
            s = s + ' ' + '\n'
        else:
            s = s + ' ' + i
        
    tag.string = s

    soup.find_all('script',type="text/javascript")[-1] = tag
    with open('./Regdoc_graphs/'+idoc+'.html', "w") as outf:
        outf.write(str(soup))
        
    with open('./Regdoc_graphs/'+idoc+'.html') as inf:
        txt = inf.read()
        soup = bs4.BeautifulSoup(txt)  
    c = soup.find_all('script',type="text/javascript")
    tag = c[-1]
    
    t=0
    start = 0
    tindex = []
    tokens1 = tag.contents[0].splitlines(True)
    
    t0=0
    while(t0<len(tokens1)):
        if('"edges": {' in tokens1[t0]):
            break
        t0+=1
        
    with open("nodefont.js") as f:
        jstext = f.read()
    
    p = []
    t=0
    while(t<t0):
        p.append(tokens1[t])
        t=t+1
    p.append(jstext)
    t = t0
    while(t<len(tokens1)):
        p.append(tokens1[t])
        t=t+1
    
    s = ''
    for i in p:
        if(i == '\n'):
            s = s + ' ' + '\n'
        else:
            s = s + ' ' + i
        
    tag.string = s
    
    soup.find_all('script',type="text/javascript")[-1] = tag
    with open('./Regdoc_graphs/'+idoc+'.html', "w") as outf:
        outf.write(str(soup))

#### Informal Guidelines

In [21]:
def informalguidance(docs, finalclean48):
    
    with open('ig.pickle','rb') as f:
        igdata = pickle.load(f)
        
    flag=0
    iggraph = []
    iglabel = []
    igdocs = []
    for i in igdata:
        regindex = i[1].split()[1]
        if(len(regindex) < 7 and ',' not in regindex):
            if('(' not in regindex and ')' in regindex):
                regindex = regindex.replace(')','')

            subt=0
            while(subt<len(regindex)):
                if(regindex[subt] == '('):
                    p=subt
                    siter=0
                    s=''
                    while(siter<p):
                        s = s + regindex[siter]
                        siter+=1
                    ri = s
                    s = ''
                    while(p<len(regindex)):
                        if(regindex[p]!=')' and regindex[p].isdigit() == True):
                            s = s + regindex[p]
                        p+=1
                    srid = s
                    break
                subt+=1
            if(srid == ''):
                srid = 'nil'
        rid = [ri,srid]
    
        dind = docs.index(i[0])
        cur_regs = finalclean48[dind]
        regind = regids(cur_regs)
    
        iditer = 0
        while(iditer<len(regind)):
            if(rid[1] != 'nil'):
                if(rid == regind[iditer]):
                    iggraph.append([cur_regs[iditer],i[2],'3'])
                    lbl = ''
                    for li in i[3]:
                        lbl = lbl + ' ' + li
                    iglabel.append(lbl)
                    igdocs.append(i[0])
            else:
                if(rid[0] == regind[0]):
                    iggraph.append([cur_regs[iditer],i[2],'3'])
                    lbl = ''
                    for li in i[3]:
                        lbl = lbl + ' ' + li
                    iglabel.append(lbl)
                    igdocs.append(i[0])
            iditer+=1
    return iggraph, iglabel, igdocs

#### Legal Cases

In [45]:
def legalcases(docs, finalclean48):
    
    with open('lc.pickle','rb') as f:
        igdata = pickle.load(f)
        
    flag=0
    iggraph = []
    iglabel = []
    igdocs = []
    for i in igdata:
        regindex = i[1].split()[1]
        if(len(regindex) < 7 and ',' not in regindex):
            if('(' not in regindex and ')' in regindex):
                regindex = regindex.replace(')','')

            subt=0
            while(subt<len(regindex)):
                if(regindex[subt] == '('):
                    p=subt
                    siter=0
                    s=''
                    while(siter<p):
                        s = s + regindex[siter]
                        siter+=1
                    ri = s
                    s = ''
                    while(p<len(regindex)):
                        if(regindex[p]!=')' and regindex[p].isdigit() == True):
                            s = s + regindex[p]
                        p+=1
                    srid = s
                    break
                subt+=1
            if(srid == ''):
                srid = 'nil'
        rid = [ri,srid]
    
        dind = docs.index(i[0])
        cur_regs = finalclean48[dind]
        regind = regids(cur_regs)
    
        iditer = 0
        while(iditer<len(regind)):
            if(rid[1] != 'nil'):
                if(rid == regind[iditer]):
                    iggraph.append([cur_regs[iditer],str(i[2]),'4'])
                    lbl = ''
                    for li in i[3]:
                        lbl = lbl + ' ' + li
                    iglabel.append(lbl)
                    igdocs.append(i[0])
            else:
                if(rid[0] == regind[0]):
                    iggraph.append([cur_regs[iditer],str(i[2]),'4'])
                    lbl = ''
                    for li in i[3]:
                        lbl = lbl + ' ' + li
                    iglabel.append(lbl)
                    igdocs.append(i[0])
            iditer+=1
    lcgraph = iggraph
    lclabel = iglabel
    lcdocs = igdocs
    return lcgraph, lclabel, lcdocs

In [55]:
iggraph, iglabel, igdocs = informalguidance(docs, finalclean48)
lcgraph, lclabel, lcdocs = legalcases(docs, finalclean48)
t=0
topiccount = 0
for idoc in docs:
    cur_regs = finalclean48[t]
    regdata = []
    regind = regids(cur_regs)
    regdata = crossref(cur_regs, regind, regdata)
    igl = []
    lcl = []
    
    d = docchaps[t]
    regdata = chapterconnect(cur_regs, d, regdata)

    topics = regtopics[topiccount:topiccount+len(cur_regs)]
    topiccount = topiccount + len(cur_regs)
    regdata = commontopics(topics,cur_regs,regdata)
    
    diter=0
    while(diter<len(igdocs)):
        if(igdocs[diter] == idoc):
            regdata.append(iggraph[diter])
            igl.append(iglabel[diter])
        diter+=1

    diter=0
    while(diter<len(lcdocs)):
        if(lcdocs[diter] == idoc):
            regdata.append(lcgraph[diter])
            lcl.append(lclabel[diter])
        diter+=1
    
    df = pd.DataFrame(regdata, columns=['source','target','label'])
    #if(idoc != 'Substantial Acquisition of Shares and Takeovers'):
    layer21(df, idoc, regind, igl, lcl, t, cur_regs)
    t=t+1

In [48]:
print(len(igdocs))
print(len(lcdocs))

657
4


#### Layer 2 (All)

YET TO DO: Submitted shared topics data to gpu

In [56]:
def layer22(df, finalclean48,iglabel):

    sources = df['source']
    targets = df['target']
    labels = df['label']
    q = 'Layer 2 - All Regulations'
    edge_data = zip(sources, targets, labels)
    igcount=0
    lccount=0
    #nt = Network(height="750px", width="100%", heading=q)
    nt = Network(height="100%", width="100%",heading=q)
    nt.hrepulsion()
    
    lbl = ['Cross-reference', 'Chapterwise', 'Shared topics']
    t=0
    for e in edge_data:
        src = e[0]
        dst = e[1]
        h = e[2]
        for i in finalclean48:
            if(src in i):
                sdocnumber = finalclean48.index(i)
                scur_regs = i
            if(dst in i):
                ddocnumber = finalclean48.index(i)
                dcur_regs = i
        if(h == '0'):
            sid = scur_regs.index(src)
            did = dcur_regs.index(dst)
            nt.add_node(src, label='sub-reg', title=src, color = "blue", url='./Reg_graphs/'+str(sdocnumber)+'_'+str(sid)+'.html')
            nt.add_node(dst, label='sub-reg', title=dst, color = "blue", url='./Reg_graphs/'+str(ddocnumber)+'_'+str(sid)+'.html')
            nt.add_edge(src, dst, title=lbl[0], value=8, color = "#000000", length=400)
        elif(h == '1'):
            sid = scur_regs.index(src)
            did = dcur_regs.index(dst)
            nt.add_node(src, label='sub-reg', title=src, color = "blue", url='./Reg_graphs/'+str(sdocnumber)+'_'+str(sid)+'.html')
            nt.add_node(dst, label='sub-reg', title=dst, color = "blue", url='./Reg_graphs/'+str(ddocnumber)+'_'+str(sid)+'.html')
            nt.add_edge(src, dst, title=lbl[1], value=7, color = "red", length=400)
        elif(h == '2'):
            sid = scur_regs.index(src)
            did = dcur_regs.index(dst)
            nt.add_node(src, label='sub-reg', title=src, color = "blue", url='./Reg_graphs/'+str(sdocnumber)+'_'+str(sid)+'.html')
            nt.add_node(dst, label='sub-reg', title=dst, color = "blue", url='./Reg_graphs/'+str(ddocnumber)+'_'+str(sid)+'.html')
            nt.add_edge(src, dst, title=lbl[2], color = "green", length=400)
        elif(h == '3'):
            sid = cur_regs.index(src)
            nt.add_node(src, label=str(docnumber)+'_'+str(sid), title=src, color = "blue", url='./Reg_graphs/'+str(docnumber)+'_'+str(sid)+'.html')
            nt.add_node(dst, label='informal_guidance', title=dst, color = "red", url='file:///home/deepti-saravanan/Desktop/knowledge_graph_SEBI/informal_guidance/'+dst.replace('.docx','')+'.pdf')
            nt.add_edge(src, dst, title=igl[igcount], color = "#000000", length=400)
            igcount = igcount+1
            if(igcount == 657):
                igcount=656
        else:
            sid = cur_regs.index(src)
            nt.add_node(src, label=str(docnumber)+'_'+str(sid), title=src, color = "blue", url='/home/deepti-saravanan/Desktop/IIITH/variational_text_inference/Reg_graphs/'+str(docnumber)+'_'+str(sid)+'.html')
            nt.add_node(dst, label='legal_case', title=dst, color = "black", url='file:///home/deepti-saravanan/Desktop/knowledge_graph_SEBI/case_pdf/'+dst+'.pdf')
            nt.add_edge(src, dst, title=lcl[lccount], color = "#000000", length=400)
            lccount = lccount+1
        t=t+1
    
    nt.show_buttons()  
    nt.write_html('./Regdoc_graphs/all.html')
    
    with open('./Regdoc_graphs/all.html') as inf:
        txt = inf.read()
        soup = bs4.BeautifulSoup(txt)  
    c = soup.find_all('script',type="text/javascript")
    tag = c[-1]

    t=0
    start = 0
    tindex = []
    tokens1 = tag.contents[0].splitlines(True)

    t=0
    appendind = []
    for i in tokens1:
        if('return' in i):
            appendind.append(t)
        t=t+1
    
    with open("interactive.js") as f:
        jstext = f.read()
    
    p = []
    t=0
    while(t<appendind[-1]):
        p.append(tokens1[t])
        t=t+1
    p.append(jstext)
    t = appendind[-1]
    while(t<len(tokens1)):
        p.append(tokens1[t])
        t=t+1
    
    s = ''
    for i in p:
        if(i == '\n'):
            s = s + ' ' + '\n'
        else:
            s = s + ' ' + i
        
    tag.string = s

    soup.find_all('script',type="text/javascript")[-1] = tag
    with open('./Regdoc_graphs/all.html', "w") as outf:
        outf.write(str(soup))
        
    with open('./Regdoc_graphs/all.html') as inf:
        txt = inf.read()
        soup = bs4.BeautifulSoup(txt)  
    c = soup.find_all('script',type="text/javascript")
    tag = c[-1]
    
    t=0
    start = 0
    tindex = []
    tokens1 = tag.contents[0].splitlines(True)
    
    t0=0
    while(t0<len(tokens1)):
        if('"edges": {' in tokens1[t0]):
            break
        t0+=1
        
    with open("nodefont.js") as f:
        jstext = f.read()
    
    p = []
    t=0
    while(t<t0):
        p.append(tokens1[t])
        t=t+1
    p.append(jstext)
    t = t0
    while(t<len(tokens1)):
        p.append(tokens1[t])
        t=t+1
    
    s = ''
    for i in p:
        if(i == '\n'):
            s = s + ' ' + '\n'
        else:
            s = s + ' ' + i
        
    tag.string = s
    
    soup.find_all('script',type="text/javascript")[-1] = tag
    with open('./Regdoc_graphs/all.html', "w") as outf:
        outf.write(str(soup))

In [57]:
iggraph, iglabel, igdocs = informalguidance(docs, finalclean48)
reggraphdata.extend(iggraph)
df = pd.DataFrame(reggraphdata, columns=['source','target','label'])
layer22(df,finalclean48,iglabel)

KeyboardInterrupt: 

#### Layer 3

In [421]:
import pickle5 as pickle
with open('48annotated.pkl','rb') as f:
    regtags = pickle.load(f)

In [457]:
def indreg(regtags, finalclean48):
    d=41
    while(d<len(finalclean48)):
        print(d)
        regz = finalclean48[d]
        reg = list(regtags[d].keys())
        labels = list(regtags[d].values())
        
        li=0
        while(li<len(labels)):
            l = labels[li]
            r = reg[li]
            
            wordlabels1 = []
            for tag in l:
                k  = tag['start']
                w = ''
                while(k<tag['end']):
                    w = w + r[k]
                    k=k+1
                wordlabels1.append([w,tag['label']])
            
            for j in wordlabels1:
                if(j[1] == 'ADJ'):
                    if(j[0] != 'such'):
                        wordlabels1.remove(j)
            
            for j in wordlabels1:
                t=0
                if(t!=0):
                    if(j[1] == 'VERB'):
                        prevword = wordlabels1[t-1]
                        if(j[0] in prevword[0]):
                            wordlabels1.remove(j)
                t=t+1
            w = [wordlabels1]
            
            rsource = ['empty']
            rtarget = ['empty']
            rlabel = ['empty']
            nodes = ['Person - General','Object','Person In charge','time','subject','PER','Assets','Investigation','Legal Doc','Company','ORG','Penalty']
            cpt=0
            for wordlabels in w:
                k=0
                p=-1
                samecounter = 0
                while(k<len(wordlabels)):
                    if(k>p):
                        i = wordlabels[k]
            
                        #Non-noun words
            
                        if(i[1] not in nodes):
                
                            #Source node identification
                
                            t = len(rtarget)
                            s = rtarget[t-1]
                            if((i[0] == 'or' or i[0] == 'and') and k+1 < len(wordlabels)):
                                after = wordlabels[k+1]
                                if(after[1] in nodes):
                                    s = rsource[-1]
                                else:
                                    s = rsource[-1] 
                            rsource.append(s)
                            samecounter = samecounter+1
                
                            #Target node and edge label identification
                
                            p=k
                            l = ''
                            while(p<len(wordlabels)):
                                j = wordlabels[p]
                                if(j[1] not in nodes):
                                    if(j[0] != 'such'):
                                        l = l + ' ' + j[0]
                                    if(j[0] == 'or' or j[0] == 'and'):
                                        prev = wordlabels[p-1]
                                        if(prev[1] in nodes):
                                            l = rlabel[-1]
                                    if(j[0] == 'same'):
                                        samewords = []
                                        rs = rsource[len(rsource)-samecounter-1:]
                                        for stoken in rs:
                                            for token in wordlabels:
                                                if(token[0] == stoken):
                                                    if(token[1] == 'Object'):
                                                         samewords.append(token[0])
                                            if(len(samewords) != 0):
                                                rlabel.append(l)
                                                rtarget.append(samewords[0]) 
                                                break
                                            else:
                                                continue
                            
                                    if(j[0] == 'such' and p+1 < len(wordlabels)):
                                        nextword = wordlabels[p+1]
                                        for st in rsource:
                                            if(nextword[0] in st):
                                                targetword = st
                                                break
                                            else:
                                                targetword = nextword[0]
                                        rtarget.append(targetword)
                                        rlabel.append(l)
                            
                                        if(cpt==1):
                                            s = rtarget[-3]
                                            rsource.append(s)
                                            rlabel.append(l)
                                            rtarget.append(j[0])
                                            cpt=0
                            
                                        prev = wordlabels[p-1]
                                        if(prev[0] == 'or' or prev[0] == 'and'):
                                            cpt=1
                                        p=p+1
                                        break
                                else:
                                    rtarget.append(j[0])
                                    rlabel.append(l)
                        
                                    if(cpt==1):
                                        s = rtarget[-3]
                                        rsource.append(s)
                                        rlabel.append(l)
                                        rtarget.append(j[0])
                                        cpt=0
                        
                                    prev = wordlabels[p-1]
                                    if(prev[0] == 'or' or prev[0] == 'and'):
                                        cpt=1
                                    break
                                p=p+1
                    
                        #Nouns
            
                        else:
                
                            #Source node identification
                            if(k==0):
                                prev = 'nil'
                            else:
                                prev = wordlabels[k-1]
                            #print(prev)
                            if(prev[1] in nodes):
                                orloc = k
                                if(orloc+5 < len(wordlabels)):
                                    orloc1 = orloc+5
                                else:
                                    orloc1 = len(wordlabels)
                                orwords = []
                                t = len(rsource)
                                s = rsource[t-1]
                                l = rlabel[t-1]
                                while(orloc < orloc1):
                                    orloc2 = wordlabels[orloc]
                                    #print(orloc2)
                                    orwords.append(orloc2[0])
                                    orloc+=1
                                if('or' in orwords):
                                    orind = orwords.index('or')
                                    orloc = 0
                                    k1 = k
                                    while(orloc < orind):
                                        if(k1 < len(wordlabels)-1):
                                            orloc2 = wordlabels[k1]
                                        else:
                                            orloc2 = wordlabels[-1]
                                        rsource.append(s)
                                        samecounter = samecounter+1
                                        rlabel.append(l)
                                        rtarget.append(orloc2[0])
                                        k1+=1
                                        orloc+=1
                                    rsource.append(s)
                                    samecounter = samecounter+1
                                    rlabel.append(l)
                                    k1+=1
                                    if(k1 < len(wordlabels)-1):
                                        orloc2 = wordlabels[k1]
                                    else:
                                        orloc2 = wordlabels[-1]
                                    rtarget.append(orloc2[0])
                                    p = k1+1
                        
                                elif('and' in orwords):
                                    orind = orwords.index('and')
                                    orloc = k
                                    while(orloc < orind):
                                        orloc2 = wordlabels[orloc]
                                        rsource.append(s)
                                        samecounter = samecounter+1
                                        rlabel.append(l)
                                        rtarget.append(orloc2[0])
                                        orloc+=1
                                    rsource.append(s)
                                    samecounter = samecounter+1
                                    rlabel.append(l)
                                    orloc+=1
                                    orloc2 = wordlabels[orloc]
                                    rtarget.append(orloc2[0])
                                    p = orloc
                            
                                else:
                                    l = ''
                                    t = len(rsource)
                                    s = rtarget[t-1]
                                    rsource.append(s)
                                    samecounter = samecounter+1
                                    rtarget.append(i[0])
                                    rlabel.append('nil')
                            else:
                                l = ''
                                if(k==0):
                                    s1 = wordlabels[k]
                                    s=s1[0]
                                else:
                                    t = len(rsource)
                                    s = rtarget[t-1]
                                rsource.append(s)
                                samecounter = samecounter+1
                
                                if(k==0):
                                    p=k+1
                                else:
                                    p=k
                                while(p<len(wordlabels)):
                                    j = wordlabels[p]
                                    if(j[1] not in nodes):
                                        if(j[0] != 'such'):
                                            l = l + ' ' + j[0]
                                        if(j[0] == 'or' or j[0] == 'and'):
                                            prev = wordlabels[p-1]
                                            if(prev[1] in nodes):
                                                l = rlabel[-1]
                                        if(j[0] == 'same'):
                                            samewords = []
                                            rs = rsource[len(rsource)-samecounter-1:]
                            
                                            for stoken in rs:
                                                for token in wordlabels:
                                                    if(token[0] == stoken):
                                                        if(token[1] == 'Object'):
                                                             samewords.append(token[0])
                                            if(len(samewords) != 0):
                                                rlabel.append(l)
                                                rtarget.append(samewords[0]) 
                                                break
                                            else:
                                                continue
                                        if(j[0] == 'such' and p+1 < len(wordlabels)):
                                            nextword = wordlabels[p+1]
                                            for st in rsource:
                                                if(nextword[0] in st):
                                                    targetword = st
                                                    break
                                                else:
                                                    targetword = nextword[0]
                                            rtarget.append(targetword)
                                            rlabel.append(l)
                            
                                            if(cpt==1):
                                                s = rtarget[-3]
                                                rsource.append(s)
                                                samecounter = samecounter+1
                                                rlabel.append(l)
                                                rtarget.append(j[0])
                                                cpt=0
                            
                                            prev = wordlabels[p-1]
                                            if(prev[0] == 'or' or prev[0] == 'and'):
                                                cpt=1
                                            p=p+1
                                            break
                                    else:
                                        rtarget.append(j[0])
                                        rlabel.append(l)
                        
                                        if(cpt==1):
                                            s = rtarget[-3]
                                            rsource.append(s)
                                            samecounter = samecounter+1
                                            rlabel.append(l)
                                            rtarget.append(j[0])
                                            cpt=0
                            
                                        prev = wordlabels[p-1]
                                        if(prev[0] == 'or' or prev[0] == 'and'):
                                            cpt=1
                                        break
                                    p=p+1
                        if(len(rsource) != len(rtarget)):
                            del rsource[len(rsource)-1]
                    k=k+1
                   
            ingreggraph = []       
            if(rlabel[0] == 'empty'):
                rlabel.remove('empty')
                rtarget.remove('empty')
                rsource.pop(0)   
            
            regiter = 0
            while(regiter<len(rsource)):
                ingreggraph.append([rsource[regiter], rtarget[regiter], rlabel[regiter]])
                regiter+=1
            df = pd.DataFrame(ingreggraph, columns=['source','target','label'])
            
            layer3(df,regz[li],d,li)
            #print(li)
            li+=1
        d+=1

In [411]:
def layer3(df, reg, d,li):
    sources = df['source']
    targets = df['target']
    labels = df['label']
    q = 'Layer 3 - ' + reg
    edge_data = zip(sources, targets, labels)
    
    #nt = Network(height="750px", width="100%", heading=q)
    nt = Network(height="100%", width="100%",heading=q, directed=True)
    nt.hrepulsion()
    t=0
    for e in edge_data:
        src = e[0]
        dst = e[1]
        lbl = e[2]
        if(t==0):
            nt.add_node(src, label=src, title=src, color = "red")
            nt.add_node(dst, label=dst, title=dst, color = "blue")
            nt.add_edge(src, dst, title=lbl, color = "#000000")
        else:
            nt.add_node(src, label=src, title=src, color = "blue")
            nt.add_node(dst, label=dst, title=dst, color = "blue")
            nt.add_edge(src, dst, title=lbl, color = "#000000")
        t=t+1
    nt.show_buttons()  
    nt.write_html('./Reg_graphs/'+str(d)+'_'+str(li)+'.html')
        
    with open('./Reg_graphs/'+str(d)+'_'+str(li)+'.html') as inf:
        txt = inf.read()
        soup = bs4.BeautifulSoup(txt)  
    c = soup.find_all('script',type="text/javascript")
    tag = c[-1]
    
    t=0
    start = 0
    tindex = []
    tokens1 = tag.contents[0].splitlines(True)
    
    t0=0
    while(t0<len(tokens1)):
        if('"edges": {' in tokens1[t0]):
            break
        t0+=1
        
    with open("/home/deepti-saravanan/Desktop/laws/nodefont.js") as f:
        jstext = f.read()
    
    p = []
    t=0
    while(t<t0):
        p.append(tokens1[t])
        t=t+1
    p.append(jstext)
    t = t0
    while(t<len(tokens1)):
        p.append(tokens1[t])
        t=t+1
    
    s = ''
    for i in p:
        if(i == '\n'):
            s = s + ' ' + '\n'
        else:
            s = s + ' ' + i
        
    tag.string = s
    
    soup.find_all('script',type="text/javascript")[-1] = tag
    with open('./Reg_graphs/'+str(d)+'_'+str(li)+'.html', "w") as outf:
        outf.write(str(soup))

In [458]:
indreg(regtags, finalclean48)

41
42
43
44
45
46
47
